In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
import random # use it for random crawling automately 
import time

In [4]:
from tqdm import tqdm, trange
# Check the progress when getting bid data crawling 

In [5]:
import re
#regular expression

In [ ]:
url= "https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum=1&menuNo=300102&cateCd=&searchText=&status1Cd=&Status2Cd=&sugMonthTp=201906&orderKey=registDtDesc"

In [ ]:
# get html code 
response = requests.get(url)
response
# <Response [200]> 200 = there is page / 404 = no page

In [ ]:
def get_suggestion_list(pnum):
    # print(f'year_month : {year_month}, pnum: {pnum}')
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300102&cateCd=&searchText=&status1Cd=&Status2Cd=&sugMonthTp=201906&orderKey=registDtDesc"
    # base_url 페이지를 받아온다.
    response = requests.get( base_url )
    # status_code 가 200 OK 일 때 아래의 코드를 실행한다.
    if response.status_code == 200:
        # 웹페이지의 내용을 가져왔다면 뷰티풀숩을 통해 html 태그를 가져온다.
        html = bs(response.text, 'html.parser')
        # html 태그 내용 중 아래의 selector를 통해 목록에 있는 다른 게시글을 가져오도록 한다.
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        # tags 의 내용이 없다면 suggestion_list 를 반환한다.
        if not tags :
            return(suggestion_list)
        else :
            # tags 에서 다른 게시글을 가져와 suggestion_list 에 담아준다.
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        # 페이지번호를 하나씩 더해서 재귀호출을 합니다.
        get_suggestion_list(pnum)
        print(pnum)
    else:
        return(suggestion_list)

In [ ]:
# 목록 리스트에 있는 url를 넘겨주면 내용을 크롤링 해옵니다.
def get_suggestion_content(last_webpage):
    article = []
    
    sgId = int(last_webpage.split('=')[2])
    for id_num in tqdm(range(1, sgId +1)):
        records = []
        url = "https://www.innogov.go.kr/ucms/ogp/sug/view.do?menuNo=300102&sgId={0}".format(id_num)
        response = requests.get(url)
        html = bs(response.text, 'html.parser')
        if len(html.select('#content > div.vveiw_box1 > dl > dt')) > 0:
            # 제목을 가져옵니다.
            html.select('#content > div.vveiw_box1 > dl > dt')
            title = html.select('#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
            # dd 태그에 있는 내용을 가져와서 카테고리, 시작일, 마감일, 제안인 정보를 추출합니다.
            desc = html.select(
                '#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
            category = desc[0].get_text(strip=True)
            content = html.select(
                '#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
            #content > div.vveiw_box1 > div.vveiw_cont > div > pre > p
            start = desc[1].get_text(strip=True)
            end = desc[2].get_text(strip=True)
            author = desc[3].get_text(strip=True)
            # 추천인원을 가져옵니다.
            vote = html.select('#counter')[0].get_text(strip=True)
            #counter
     

            records.append(id_num)
            records.append(title)
            records.append(category)
            records.append(content)
            records.append(start)
            records.append(end)
            records.append(vote)
            records.append(author)
            article.append(records)

    return article

In [ ]:
# check the last page 
last_webpage = "https://www.innogov.go.kr/ucms/ogp/sug/view.do?menuNo=300102&sgId=217"
sgId = int(last_webpage.split('=')[2])


In [ ]:
# Need to get last page
article = []
for id_num in tqdm(range(1,sgId + 1)):
    records = []
    url = "https://www.innogov.go.kr/ucms/ogp/sug/view.do?menuNo=300102&sgId={0}".format(id_num)
    response = requests.get(url)
    html = bs(response.text, 'html.parser')
    if len(html.select('#content > div.vveiw_box1 > dl > dt')) > 0:
        # 제목을 가져옵니다.
        html.select('#content > div.vveiw_box1 > dl > dt')
        title = html.select('#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        # dd 태그에 있는 내용을 가져와서 카테고리, 시작일, 마감일, 제안인 정보를 추출합니다.
        desc = html.select('#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select('#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        #content > div.vveiw_box1 > div.vveiw_cont > div > pre > p
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        # 추천인원을 가져옵니다.
        vote = html.select('#counter')[0].get_text(strip=True)
        #counter
        records.append(id_num)
        records.append(title)
        records.append(category)
        records.append(content)
        records.append(start)
        records.append(end)
        records.append(vote)
        records.append(author)
        article.append(records)

In [ ]:
import pandas as pd
df_suggestion=pd.DataFrame(article)
columns = ["sgID","title","category","content", "start", "end", "vote","author"]
df_suggestion.columns = columns

In [ ]:
# save suggesion file as csv
df_suggestion.to_csv("df_suggestion")

In [35]:
# load suggestion data
df_suggestion = pd.read_csv("df_suggestion", index_col = 0) 

In [39]:
df_suggestion.describe()['vote']

count     178.000000
mean       43.466292
std       218.796481
min         0.000000
25%         2.000000
50%         4.000000
75%         7.000000
max      2037.000000
Name: vote, dtype: float64

In [42]:
# Hot topic
df_suggestion.loc[df_suggestion['vote'] == 2037]

,sgID,title,category,content,start,end,vote,author
56,64,난임지원확대 첫아이 횟수패지,보건복지,요즘 뉴스에 심심치않게 저출산 관련내용이많이나옵니다정부에선 많은 계획을말했지만 정작...,2018.10.02,2018.12.30,2037,엄마란소리
